# Importing packages

In [1]:
import sys
sys.path.append('/home/gasanoe/fedmix_on_fedjax/src/')

In [2]:
from absl import app

import fedjax
from fedjax.core import tree_util

import jax
import jax.numpy as jnp

import PLM
import itertools
import FedMix

2021-10-30 20:22:50.911838: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory


In [5]:
import pickle

# GridSearch

In [6]:
train_fd, test_fd = fedjax.datasets.emnist.load_data(only_digits=False, cache_dir='../data/')

Reusing cached file '../data/federated_emnist_train.sqlite'
Reusing cached file '../data/federated_emnist_test.sqlite'


In [7]:
model = fedjax.models.emnist.create_conv_model(only_digits=False)

In [ ]:
# fedjax.training.set_tf_cpu_only()

In [8]:
fedjax.set_for_each_client_backend('pmap')

In [9]:
def loss(params, batch, rng):
    # `rng` used with `apply_for_train` to apply dropout during training.
    preds = model.apply_for_train(params, batch, rng)
    # Per example loss of shape [batch_size].
    example_loss = model.train_loss(batch, preds)
    return jnp.mean(example_loss)

In [10]:
grad_fn = jax.jit(jax.grad(loss))

In [11]:
def loss_for_eval(params, batch):
    preds = model.apply_for_eval(params, batch)
    example_loss = model.train_loss(batch, preds)
    return jnp.mean(example_loss)

In [12]:
grad_fn_eval = jax.jit(jax.grad(loss_for_eval))

In [13]:
client_ids = train_fd.client_ids()

In [14]:
PLM_dict = {}
alphas_dict = {}
alpha = 0.5
for client_id in client_ids:
    alphas_dict[client_id] = jnp.asarray(alpha)

In [15]:
save_plm_file = '../results/PLM_EMNIST.pickle'

In [16]:
with open(save_plm_file, 'rb') as handle:
    PLM_dict = pickle.load(handle)

In [17]:
# lrs = 10**jnp.arange(-5, 1.5, 0.5)
# batch_sizes = [10, 20, 50, 100]

In [18]:
# for debugging we set a small grid
lrs = [0.1]
batch_sizes = [10]

In [19]:
num_clients_per_round=2
max_rounds = 100

In [20]:
GridSearch_table_fedmix = jnp.zeros(shape=(len(lrs), len(batch_sizes)))

In [21]:
client_batch_hparams_eval = fedjax.BatchHParams(batch_size=256)

In [23]:
for lr_id, lr in enumerate(lrs):
    print('Learning rate = {}'.format(lr))
    for b_id, batch_size in enumerate(batch_sizes):
        print('Batch size = {}'.format(batch_size))
        server_optimizer = fedjax.optimizers.adam(learning_rate=lr, b1=0.9, b2=0.999, eps=10**(-4))
        train_client_sampler = fedjax.client_samplers.UniformGetClientSampler(fd=train_fd, num_clients=num_clients_per_round, seed=0)
        client_batch_hparams = fedjax.ShuffleRepeatBatchHParams(batch_size=batch_size, num_steps=1)
        algorithm = FedMix.fedmix(grad_fn, server_optimizer, client_batch_hparams, PLM_dict, alphas_dict)
        init_params = model.init(jax.random.PRNGKey(11))
        server_state = algorithm.init(init_params)
        for round_num in range(max_rounds):
            print('Round {} / {}'.format(round_num + 1, max_rounds))
            clients = train_client_sampler.sample()
            print(clients)
            server_state, _ = algorithm.apply(server_state, clients)
        clients = train_client_sampler.sample()
        client_data_for_evaluation = [(alphas_dict[cid], PLM_dict[cid], cds) for cid, cds, _ in clients]
        grid_search_metrics = FedMix.evaluate_model(model, server_state.params, client_data_for_evaluation, client_batch_hparams_eval)
        GridSearch_table_fedmix[lr_id, b_id] = grid_search_metrics['accuracy']

Learning rate = 0.1
Batch size = 10
Debug point 3
Round 1 / 100
[(b'56958d208b65e338:f2010_38', <fedjax.core.client_datasets.ClientDataset object at 0x1510f2daf100>, DeviceArray([4146024105,  967050713], dtype=uint32)), (b'973fa7178a71bc5a:f2358_71', <fedjax.core.client_datasets.ClientDataset object at 0x1510f2daf460>, DeviceArray([2718843009, 1272950319], dtype=uint32))]
Debug point 1
server_state.params type= <class 'haiku._src.data_structures.FlatMapping'>
<class 'fedjax.core.client_datasets.ShuffleRepeatBatchView'>
<class 'haiku._src.data_structures.FlatMapping'>
Debug point 4


ValueError: Custom node type mismatch: expected type: <class 'haiku._src.data_structures.FlatMapping'>, value: None.

In [ ]:

for round_num in range(1, 3):
# Sample 10 clients per round without replacement for training.
    clients = train_client_sampler.sample()
    print('debug')
    # Run one round of training on sampled clients.
    server_state, client_diagnostics = algorithm.apply(server_state, clients);
    # print(f'[round {round_num}]')
    # Optionally print client diagnostics if curious about each client's model
    # update's l2 norm.
    print(f'[round {round_num}] client_diagnostics={client_diagnostics}')